Notebook that trains an LSTM to generate tweets based on a given emojis

First we load the data

In [26]:
import numpy as np
import pandas as pd
import data_load_utils as util

tweets = util.filter_tweets_min_count(
    util.read_tweet_data('data/emojis_homemade.csv'),
    min_count=1000)

tweets['text'] = util.filter_text_for_handles(tweets['text'])


/home/nickdbn/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2802: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [27]:
tweets

       Unnamed: 0                                               text  emoji
0               0  RT [VID] 181023 - Foi adicionada a letra “D” n...      ©
1               1  RT 181023 Kris Wu Studio update (3/3)\n\nLegen...      💫
2               2  RT Now you are watching Indian SuperStar with ...      😎
3               3                                    dats for keeps       💛
6               6                             Holy shit no I think.       😩
7               7                  RT army, follow who retweet this       👑
8               8  RT Simply K-Pop harddrive dump\n\n#베리굿 #BerryG...      🤩
9               9  Happy birthday nellie hope you have a fantasti...      ❤
10             10  RT "I have passed through fire"\n\nThank you f...      💕
12             12  RT Incredible to be involved in the making of ...      😱
13             13  RT Too much feels \n\n#BTS #MPN #BTSARMY https...      😢
14             14  RT #PL TOP SCORERS \n\n7 - \n6 - the goods  ht...      🔥
15          

In [28]:
tweets.iloc[0,:]

Unnamed: 0                                                    0
text          RT [VID] 181023 - Foi adicionada a letra “D” n...
emoji                                                         ©
Name: 0, dtype: object

In [29]:
tweets.loc[:,'text', 'emoji']

IndexingError: Too many indexers

In [30]:
tweets.loc[:,['text', 'emoji']]

                                                     text  emoji
0       RT [VID] 181023 - Foi adicionada a letra “D” n...      ©
1       RT 181023 Kris Wu Studio update (3/3)\n\nLegen...      💫
2       RT Now you are watching Indian SuperStar with ...      😎
3                                         dats for keeps       💛
6                                  Holy shit no I think.       😩
7                       RT army, follow who retweet this       👑
8       RT Simply K-Pop harddrive dump\n\n#베리굿 #BerryG...      🤩
9       Happy birthday nellie hope you have a fantasti...      ❤
10      RT "I have passed through fire"\n\nThank you f...      💕
12      RT Incredible to be involved in the making of ...      😱
13      RT Too much feels \n\n#BTS #MPN #BTSARMY https...      😢
14      RT #PL TOP SCORERS \n\n7 - \n6 - the goods  ht...      🔥
15      [DEPASCAL] RUSSEL HOODIE BLUE\n https://t.co/C...      🔗
16      The joys of having a make up artist as your be...      😂
18                      R

In [47]:
my_dict = {'text': "red and yellow and pink and green, orange and purple and blue, I can sing a rainbow, sing a rainbow, sing a rainbow too",
               'emoji': ":rainbow:"}
my_series = pd.Series(my_dict)

In [48]:
my_series

text     red and yellow and pink and green, orange and ...
emoji                                            :rainbow:
dtype: object

In [170]:
from importlib import reload
util = reload (util)

In [171]:
series, next_chars = util.get_series_data_from_tweet (my_series)

In [172]:
chars, char_indices = util.get_unique_chars_list(series)

Unique chars:  21


In [173]:
char_indices['c']

5

In [174]:
x, y = util.get_x_y_bool_arrays (series, next_chars)

Unique chars:  21


In [175]:
# and convert back to human-readable as a debug check:
print (util.x_y_bool_array_to_sentence(x, y, chars))

red and yellow and pink and green, orange


In [156]:
x = np.zeros ((len(series), 40, len(chars)), dtype=np.bool)
y = np.zeros ((len(series), len(chars)), dtype=np.bool)
for i, sentence in enumerate(series):
    for t, char in enumerate(sentence):
        x[i, t, unique[char]] = 1
    y[i, unique[next_chars[i]]] = 1

In [138]:
def bool_array_to_char(bool_array, chars):
    # chars is char index
    return (chars[np.argmax(bool_array.astype(int))])


def decode_line(x, chars):
    string = []
    for i in range (x.shape[0]):
#        string.append(chars[np.argmax(x[i].astype(int))])
        string.append(bool_array_to_char(x[i], chars))
    return (string)
                                 
def decode_example(x, y): 
    # decodes x, y from array type back into english
    print (''.join(decode_line(x, chars)), ":", # decode x
       bool_array_to_char(y, chars))   # decode y


In [139]:
for i in range (10):
    decode_example(x[i], y[i])

red and yellow and pink and green, orang : e
 and yellow and pink and green, orange a : n
d yellow and pink and green, orange and  : p
ellow and pink and green, orange and pur : p
ow and pink and green, orange and purple :  
and pink and green, orange and purple an : d
 pink and green, orange and purple and b : l
nk and green, orange and purple and blue : ,
and green, orange and purple and blue, I :  
 green, orange and purple and blue, I ca : n
